In [2]:
import pandas as pd

In [4]:
data = pd.read_csv('model_input_data/from_combined/com_original_data.csv')
data.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,WHIP,LOB,ERA,FIP,...,vs_PF,vs_H_A,vs_WLS,vs_PITCHER_INNING,vs_PITCHER_ER,vs_HITTER_AB,vs_HITTER_HIT,wls,vs_wls,YEAR
0,20160401HHLG0,20160401,LG,HH,0,B,1.333,0.750,3.000,7.333,...,939,0,0,11.333,4,46,13,W,L,2016
1,20160401HHLG0,20160401,HH,LG,0,T,1.147,0.690,3.176,23.059,...,939,1,1,12.000,4,42,9,L,W,2016
2,20160401HTNC0,20160401,NC,HT,0,B,1.222,0.755,4.000,22.778,...,1035,0,0,8.000,5,34,8,W,L,2016
3,20160401HTNC0,20160401,HT,NC,0,T,1.750,0.804,5.625,38.750,...,1035,1,1,9.000,4,30,9,L,W,2016
4,20160401KTSK0,20160401,KT,SK,0,T,1.111,0.698,4.000,11.444,...,971,1,0,9.000,8,36,10,W,L,2016


# 1. 연도-팀별 key(24)경기 기준 통계
#### 이때, 데이터 늘리기를 적용
- train, test를 미리 나누어 둔 후, train 부분에 대해서만 데이터 늘리기 적용

In [20]:
data_col = list(data.columns)
team = list(data.T_ID.unique())

default_col = ['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID',"YEAR"]
size_col = ['wls','vs_wls']

sum_col = ['HEADER_NO',
           'PITCHER_INNING', 'PITCHER_ER', 'HITTER_AB','HITTER_HIT',  
           'vs_PITCHER_INNING', 'vs_PITCHER_ER','vs_HITTER_AB', 'vs_HITTER_HIT']

tmp = set(default_col)|set(size_col)|set(sum_col)
mean_col = list(set(data_col)-tmp)

In [21]:
size_df = data[default_col + size_col]
sum_df = data[default_col + sum_col]
mean_df = data[default_col + mean_col]

###  1) size

In [90]:
def make_pct(df):
    tmp = pd.DataFrame({"COUNT":df.groupby(["T_ID","wls","YEAR"]).size()}).pivot_table("COUNT",["T_ID","YEAR"],"wls").reset_index().sort_values(["YEAR","T_ID"]).reset_index(drop=True)
    tmp.fillna(0,inplace= True)

    if "L" not in list(tmp.columns):
        tmp["L"] = 0

    if "W" not in list(tmp.columns):
        tmp["W"] = 0
    
    tmp["PCT"] = tmp['W']/(tmp['W']+tmp['L'])
    return tmp

In [95]:
size_df_cal = pd.DataFrame([])

key = 24
for y in [2016,2017,2018,2019,2020]:
    print("===",y)
    
    y_tmp = size_df[size_df["YEAR"]==y]
    for t in team:
        print(t)
        
        idx = 0
        
        ttmp_df = y_tmp[y_tmp["T_ID"]==t]
        tmp_size = len(ttmp_df)
        
        #test
        test_tmp_df = ttmp_df.iloc[tmp_size-key:,:]
        tmp2 = make_pct(test_tmp_df)
        tmp2['IDX'] = 777
        tmp2['MERGE_IDX'] = 0

        size_df_cal = pd.concat([size_df_cal, tmp2], axis = 0)
        
        
        # train
        tmp_df = ttmp_df.iloc[:tmp_size-key,:]
        t_size = len(tmp_df)
               
        m_idx = 1
        while idx < key:
            k_idx = 1
            for k in range((t_size-idx)//key):
                tmp1 = tmp_df.iloc[key*k+idx:key*(k+1)+idx,:]
                tmp2 = make_pct(tmp1)
                
                tmp2['IDX'] = k_idx 
                tmp2['MERGE_IDX'] = m_idx
                k_idx += 1
                m_idx += 1
                
                size_df_cal = pd.concat([size_df_cal, tmp2], axis = 0)
            idx += 1
            

size_df_cal.drop(["W","D","L"],axis = 1, inplace = True)

=== 2016
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2017
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2018
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2019
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2020
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS


In [96]:
size_df_cal

,T_ID,YEAR,PCT,IDX,MERGE_IDX
0,LG,2016,0.608696,777,0
0,LG,2016,0.500000,1,1
0,LG,2016,0.478261,2,2
0,LG,2016,0.375000,3,3
0,LG,2016,0.458333,4,4
...,...,...,...,...,...
0,SS,2020,0.583333,1,14
0,SS,2020,0.583333,1,15
0,SS,2020,0.583333,1,16
0,SS,2020,0.583333,1,17


### 2) sum

In [93]:
def make_sum(df):
    tmp = df.groupby(["T_ID","YEAR"]).sum().reset_index()
    
    # ERA
    tmp["era"] = tmp["PITCHER_ER"]*9/tmp["PITCHER_INNING"]
    tmp["vs_era"] = tmp["vs_PITCHER_ER"]*9/tmp["vs_PITCHER_INNING"]
    
    #AVG
    tmp['avg'] = tmp["HITTER_HIT"]/tmp["HITTER_AB"]
    tmp['vs_avg'] = tmp["vs_HITTER_HIT"]/tmp["vs_HITTER_AB"]
    
    re_col = ["T_ID","YEAR","HEADER_NO","era","avg","vs_era","vs_avg"]
#     re_col = ["T_ID","YEAR","HEADER_NO","era","avg","vs_era","vs_avg"] + vsteam_col
    return tmp[re_col]
    

In [97]:
sum_df_cal = pd.DataFrame([])

key = 24
for y in [2016,2017,2018,2019,2020]:
    print("===",y)
    
    y_tmp = sum_df[sum_df["YEAR"]==y]
    for t in team:
        print(t)
        
        idx = 0
        
        ttmp_df = y_tmp[y_tmp["T_ID"]==t]
        tmp_size = len(ttmp_df)
        
        #test
        test_tmp_df = ttmp_df.iloc[tmp_size-key:,:]
        tmp2 = make_sum(test_tmp_df)
        tmp2['IDX'] = 777
        tmp2['MERGE_IDX'] = 0

        sum_df_cal = pd.concat([sum_df_cal, tmp2], axis = 0)
        
        
        # train
        tmp_df = ttmp_df.iloc[:tmp_size-key,:]
        t_size = len(tmp_df)
               
        m_idx = 1
        while idx < key:
            k_idx = 1
            for k in range((t_size-idx)//key):
                tmp1 = tmp_df.iloc[key*k+idx:key*(k+1)+idx,:]
                tmp2 = make_sum(tmp1)
                
                tmp2['IDX'] = k_idx 
                tmp2['MERGE_IDX'] = m_idx
                k_idx += 1
                m_idx += 1
                
                sum_df_cal = pd.concat([sum_df_cal, tmp2], axis = 0)
            idx += 1

=== 2016
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2017
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2018
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2019
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2020
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS


In [98]:
sum_df_cal

,T_ID,YEAR,HEADER_NO,era,avg,vs_era,vs_avg,IDX,MERGE_IDX
0,LG,2016,0,3.649770,0.296069,4.993723,0.268530,777,0
0,LG,2016,0,5.358453,0.253317,4.555222,0.298349,1,1
0,LG,2016,0,4.715603,0.291169,4.659483,0.274605,2,2
0,LG,2016,0,5.584768,0.305842,5.148837,0.288889,3,3
0,LG,2016,0,5.551402,0.297110,5.833333,0.288942,4,4
...,...,...,...,...,...,...,...,...,...
0,SS,2020,3,3.693396,0.263694,5.003187,0.252488,1,14
0,SS,2020,3,3.778302,0.265997,5.151025,0.254975,1,15
0,SS,2020,3,4.009479,0.270338,5.193950,0.261462,1,16
0,SS,2020,3,3.685714,0.265743,5.065175,0.257783,1,17


### 3) mean

In [ ]:
def make_mean(df):
    tmp = df.groupby(["T_ID","YEAR"]).mean().reset_index()
    tmp.drop(["GDAY_DS"],axis=1,inplace = True)
    return tmp

In [99]:
mean_df_cal = pd.DataFrame([])

key = 24
for y in [2016,2017,2018,2019,2020]:
    print("===",y)
    
    y_tmp = mean_df[mean_df["YEAR"]==y]
    for t in team:
        print(t)
        
        idx = 0
        
        ttmp_df = y_tmp[y_tmp["T_ID"]==t]
        tmp_size = len(ttmp_df)
        
        #test
        test_tmp_df = ttmp_df.iloc[tmp_size-key:,:]
        tmp2 = make_mean(test_tmp_df)
        tmp2['IDX'] = 777
        tmp2['MERGE_IDX'] = 0

        mean_df_cal = pd.concat([mean_df_cal, tmp2], axis = 0)
        
        
        # train
        tmp_df = ttmp_df.iloc[:tmp_size-key,:]
        t_size = len(tmp_df)
               
        m_idx = 1
        while idx < key:
            k_idx = 1
            for k in range((t_size-idx)//key):
                tmp1 = tmp_df.iloc[key*k+idx:key*(k+1)+idx,:]
                tmp2 = make_mean(tmp1)
                
                tmp2['IDX'] = k_idx 
                tmp2['MERGE_IDX'] = m_idx
                k_idx += 1
                m_idx += 1
                
                mean_df_cal = pd.concat([mean_df_cal, tmp2], axis = 0)
            idx += 1


=== 2016
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2017
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2018
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2019
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2020
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS


In [ ]:
mean_df_cal 

# 2. 데이터 합치기

In [109]:
tmp_table = pd.merge(size_df_cal, sum_df_cal, how= 'left', left_on = ['T_ID',"YEAR","MERGE_IDX","IDX"], right_on  = ['T_ID',"YEAR","MERGE_IDX","IDX"])
merge_df = pd.merge(tmp_table, mean_df_cal, how = 'left', left_on = ['T_ID',"YEAR","MERGE_IDX","IDX"], right_on  = ['T_ID',"YEAR","MERGE_IDX","IDX"] )

# merge_df.drop(["MERGE_IDX"],axis=1,inplace = True)

In [111]:
merge_df.to_csv("model_input_data/augment_lstm_24group_1620.csv", index = False)

In [110]:
merge_df 

,T_ID,YEAR,PCT,IDX,MERGE_IDX,HEADER_NO,era,avg,vs_era,vs_avg,...,K_9,vs_ERA,H_A,vs_K_9,vs_FIP,H_9,oSLG,vs_OPS,vs_WHIP,P2_WHIP_RT
0,LG,2016,0.608696,777,0,0,3.649770,0.296069,4.993723,0.268530,...,6.299708,5.183583,0.625000,6.367167,21.210083,9.306833,0.376583,0.714208,1.588458,1.592333
1,LG,2016,0.500000,1,1,0,5.358453,0.253317,4.555222,0.298349,...,7.024500,4.564292,0.500000,7.958750,25.981417,10.525375,0.411833,0.777333,1.417625,1.958208
2,LG,2016,0.478261,2,2,0,4.715603,0.291169,4.659483,0.274605,...,6.010167,4.835125,0.500000,6.593083,18.368167,9.547667,0.370792,0.717458,1.526958,1.786958
3,LG,2016,0.375000,3,3,0,5.584768,0.305842,5.148837,0.288889,...,5.717083,5.223208,0.541667,6.863167,19.183125,10.450250,0.444000,0.800958,1.615917,2.005667
4,LG,2016,0.458333,4,4,0,5.551402,0.297110,5.833333,0.288942,...,6.221583,5.848958,0.416667,6.665708,24.602000,10.233583,0.445000,0.804208,1.644917,2.142625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4095,SS,2020,0.583333,1,14,3,3.693396,0.263694,5.003187,0.252488,...,6.343750,5.094958,0.583333,6.964333,23.619042,8.625000,0.371750,0.688375,1.373667,1.572542
4096,SS,2020,0.583333,1,15,3,3.778302,0.265997,5.151025,0.254975,...,6.218750,5.251208,0.541667,7.011208,24.400292,8.708333,0.379583,0.696542,1.402583,1.575000
4097,SS,2020,0.583333,1,16,3,4.009479,0.270338,5.193950,0.261462,...,6.072917,5.292875,0.500000,6.886208,24.303083,9.020833,0.386208,0.709542,1.416458,1.635167
4098,SS,2020,0.583333,1,17,3,3.685714,0.265743,5.065175,0.257783,...,6.385417,5.167875,0.458333,7.011208,24.108625,8.911458,0.388792,0.707208,1.407208,1.547667
